# Raindrop (might be wrong?)

In [1]:
# Raindrop arguments
dataset = 'P12'  # Choices: 'P12', 'P19', 'eICU', 'PAM'
withmissingratio = False  # Missing ratio ranges (0 to 0.5)
splittype = 'gender' # Choices: 'random', 'age', 'gender', which is Split type for P12 and P19

gender = "male" # Used for making embeddings for male or female (New for soruce/target)

reverse = False  # Reverse training groups (female/older)
feature_removal_level = 'no_removal' # Choices: 'no_removal', 'set', 'sample', Feature removal level (if splittype=random)
predictive_label = 'mortality' # Choices: 'mortality', 'LoS' , Predictive label for P12 (mortality or LoS)


In [2]:
import torch
from Raindrop.code.raindrop import raindrop_training

# raindrop args
args = {
    'dataset': dataset,
    'withmissingratio': withmissingratio,
    'splittype': splittype,
    'gender' : gender,
    'reverse': reverse,
    'feature_removal_level': feature_removal_level,
    'predictive_label': predictive_label
}

r""" Raindrop embeddings
The output of raindrop is 5 splits, where each split is just random data from the whole dataset
used to asses performance, since the datasets are already made, we will not touch it for now
and as for raincoat input, we can either choose 1 split, or merge 5 splits to make a big embedding 
"""
embeddings = raindrop_training(args)

filename = f"./embeddings/raindrop_{gender}_{dataset}_embeddings.pt"
torch.save(embeddings, filename)



Dataset used:  P12
args.dataset, args.splittype, args.reverse, args.withmissingratio, args.feature_removal_level P12 gender False False no_removal
missing ratio list [0]
Split id: 1
5309 1327 1327 5309 1327 1327
- - Run 1 - -
Stop epochs: 2, Batches/epoch: 1, Total batches: 2
Validation: Epoch 0,  val_loss:0.7093, aupr_val: 24.26, auc_val: 59.63
**[S] Epoch 0, aupr_val: 24.2650, auc_val: 59.6330 **
Training appending
Validation: Epoch 1,  val_loss:0.6986, aupr_val: 27.33, auc_val: 66.83
**[S] Epoch 1, aupr_val: 27.3311, auc_val: 66.8276 **
Validation appending
Split id: 2
5309 1327 1327 5309 1327 1327
- - Run 1 - -
Stop epochs: 2, Batches/epoch: 1, Total batches: 2
Validation: Epoch 0,  val_loss:0.6922, aupr_val: 20.16, auc_val: 61.24
**[S] Epoch 0, aupr_val: 20.1600, auc_val: 61.2444 **
Training appending
Validation: Epoch 1,  val_loss:0.6963, aupr_val: 23.27, auc_val: 65.42
**[S] Epoch 1, aupr_val: 23.2711, auc_val: 65.4157 **
Validation appending
Split id: 3
5309 1327 1327 5309 1327

# Raincoat

## New Raincoat

In order to use Raindrop, we have to Replace the CNN Encoder in RAINCOAT with it:
Currently, RAINCOAT uses a standard 1D CNN (CNN class in raincoat.py) to extract time-domain features. This assumes regularly sampled data.
We need to replace this CNN encoder with the core architecture of RAINDROP (Raindrop_v2 class in model_rd.py). This will allow RAINCOAT to handle the irregular sampling and capture the complex inter-sensor dependencies.

FOR DECODER:

- Reconstruction Logic: The reconstruction logic needs to be adjusted. Here are a few potential approaches:
Interpolation: You could use the timestamps and the generated features to interpolate the reconstructed signal at the desired time points.
- Time-Aware Decoder: Design a decoder that explicitly incorporates timestamps into its operations. For example, you might use a recurrent neural network (RNN) to process the features and timestamps sequentially, generating the output at each time step.
- Masking: If you have missing values (not all timestamps have observations), you might use masking to handle them during reconstruction.
- Loss Function: The current L1 reconstruction loss assumes a regular time series. You might need to adapt the loss function to account for irregularity, such as:
Weighted Loss: Give more weight to the observed time points during loss calculation.
Specialized Loss: Explore loss functions specifically designed for irregular time series reconstruction.

## Name this later?

In [14]:

# Loading embeddings for RainCoat
train_data = torch.load(f"data/WISDM/train_1.pt")

print("Keys in the dictionary:", train_data.keys())

print(train_data['samples'].shape)


Keys in the dictionary: dict_keys(['samples', 'labels'])
(102, 128, 3)


C:\Users\ayham\AppData\Local\Temp\ipykernel_9084\1745442675.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(f"data/WISDM/train_1.pt")


In [ ]:
# Raincoat arguments
# ========  Experiments Name ================
save_dir ='experiments_logs' # Directory containing all experiments
experiment_description = 'WISDM' # Name of your experiment (EEG, HAR, HHAR_SA, WISDM)

# ========= Select the DATASET ==============
data_path = r'./data' #Path containing dataset
dataset = 'WISDM' # Dataset of choice: (WISDM - EEG - HAR - HHAR_SA, Boiler)

# ========= Experiment settings ===============
num_runs = 1 # Number of consecutive run with different seeds
device = 'cuda' # cpu or cuda

In [ ]:
args = {
    'save_dir': save_dir,
    'experiment_description': experiment_description,
    'data_path': data_path,
    'dataset': dataset,
    'num_runs': feature_removal_level,
    'device': device
}

trainer = cross_domain_trainer(args)
trainer.train()